# Исследование надежности заемщиков

На основе полученнных данных я попробую выявить взаимосвязь между семейным поолжением, количеством детей и другими характеристиками клиентов и их платёжеспособностью. Результаты исследования можно будет использовать для построения модели кредитного скоринга. 

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.1 Заполнение пропусков

Пропущены значения в столбцах days_employed, total_income, причём количество пропущенных значений в этих столбцах одинаковое и составляет примерно 10% от общего количества значений. Пропуски могли возникнуть из-за того, что клиенты не трудоустроены официально и не указали эту информацию. Пропущенные количественные переменные лучше заменить медианным значением, т.к. среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства. Медианное значение позволяет получить более объективный результат. 

In [2]:
print(len(data[data['days_employed'].isna()])) 
print(len(data[data['total_income'].isna()])) 
print(len(data[data['days_employed'].isna()])/len(data))
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
data.info()

2174
2174
0.10099883855981417
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.2 Проверка данных на аномалии и исправления.

* days_employed - отрицательные значения, слишком точные значения. отрицательные значения я заменяю на равные им по модулю, взятые с положительным знаком, и во всём столбце отбрасываю дробную часть 
* children - отрицательные значения заменяю на положительные аналогично days_employed, слишком большие значения заменяю медианным
* dob_years - нулевые значения. заменяю медианным

Мне кажется, что выявленные в этих столбцах ошибки больше всего похожи на ошибки пользователей (опечатки) или ошибки разработки (в ячейках указан неверный тип данных)

In [3]:
# обработка значений трудового стажа

rows = (data['days_employed'] < 0)
data.loc[rows, 'days_employed'] = data['days_employed']*(-1)
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
data['days_employed'] = data['days_employed'].astype(int)

In [4]:
# обработка количества детей

rows = (data['children']>5) 
data.loc[rows, 'children'] = data['children'].median()
rows = (data['children']<0)
data.loc[rows, 'children'] = data['children']*(-1)
data['children'].value_counts()
data['children'] = data['children'].astype(int)

In [5]:
# обработка возраста

rows = (data['dob_years'] == 0)
data.loc[rows, 'dob_years'] = data['dob_years'].median()

In [6]:
for column in data:
    try:
        print("В столбце",column,data[data[column]<0][column].count(),"отрицательных значения")
       
    except:
        print("Столбец",column,'не числовой столбец')

В столбце children 0 отрицательных значения
В столбце days_employed 0 отрицательных значения
В столбце dob_years 0 отрицательных значения
Столбец education не числовой столбец
В столбце education_id 0 отрицательных значения
Столбец family_status не числовой столбец
В столбце family_status_id 0 отрицательных значения
Столбец gender не числовой столбец
Столбец income_type не числовой столбец
В столбце debt 0 отрицательных значения
В столбце total_income 0 отрицательных значения
Столбец purpose не числовой столбец


### Шаг 2.3. Изменение типов данных.

In [7]:
data['total_income'] = data['total_income'].astype(int)

### Шаг 2.4. Удаление дубликатов.

Если в данных присутствуют строки-дубликаты, удалите их. Также обработайте неявные дубликаты. Например, в столбце education есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к одному регистру. Проверьте остальные столбцы.
После удаления дубликатов сделайте следующее:

    поясните, как выбирали метод для поиска и удаления дубликатов в данных;
    приведите возможные причины появления дубликатов.

Для поиска дубликатов я использовала метод value_counts, там, где встречаются значения, которые отличаются только заглавными и строчными буквами, привела их к одному регистру. Дубликаты могли возникнуть из-за опечаток при ручном вводе, особенно если не были заранее оговорены требования к заполнению таблицы

In [8]:
data['education'] = data['education'].str.lower()
print(data['education'].value_counts())
print(data['family_status'].value_counts())
print(data['income_type'].value_counts())
print(data['purpose'].value_counts())

data = data.drop_duplicates()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64
женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64
свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                       

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [9]:
new_data_1 = data[['education', 'education_id']].copy()
new_data_2 = data[['family_status', 'family_status_id']].copy()
new_data_1.drop_duplicates()
new_data_2.drop_duplicates()
data = data.drop(columns = ['education', 'family_status'], axis = 1)


### Шаг 2.6. Категоризация дохода.

In [10]:
def total_income_category(income):
    
    if income<= 30000:
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    return 'A'

data['total_income_category'] = data['total_income'].apply(total_income_category)
data

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43.0,1,1,F,компаньон,0,224791,операции с жильем,B
21521,0,343937,67.0,1,0,F,пенсионер,0,155999,сделка с автомобилем,C
21522,1,2113,38.0,1,1,M,сотрудник,1,89672,недвижимость,C
21523,3,3112,38.0,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B


### Шаг 2.7. Категоризация целей кредита.

In [11]:
def purpose_category (row):
    purpose = row['purpose']
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'жил' in purpose or 'движ' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    return 'получение образования'
data['purpose_category'] = data.apply(purpose_category, axis=1)
data


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43.0,1,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21521,0,343937,67.0,1,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21522,1,2113,38.0,1,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21523,3,3112,38.0,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем


### Ответы на вопросы.

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [12]:
data_children = data.pivot_table(index=['children'], values='debt', aggfunc=['sum', 'count'])
data_children['share']=data_children['sum']/data_children['count']
data_children

,sum,count,share
,debt,debt,
children,,,
0,1071,14166,0.075604
1,445,4855,0.091658
2,194,2052,0.094542
3,27,330,0.081818
4,4,41,0.097561
5,0,9,0.000000


**Вывод:** результат показался мне неожиданным - заёмщики, у которых один, двое или четверо детей, не погашают кредит в срок значительно чаще, чем те, у кого трое детей. Но наиболее надёжные клиенты это те, у кого детей нет совсем.


#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [13]:
data_family = data.pivot_table(index=['family_status_id'], values='debt', aggfunc=['sum', 'count'])
data_family['share']=data_family['sum']/data_family['count']
display(data_family)
new_data_2.value_counts()

,sum,count,share
,debt,debt,
family_status_id,,,
0,931,12339,0.075452
1,388,4150,0.093494
2,63,959,0.065693
3,85,1195,0.071130
4,274,2810,0.097509


family_status          family_status_id
женат / замужем        0                   12339
гражданский брак       1                    4150
Не женат / не замужем  4                    2810
в разводе              3                    1195
вдовец / вдова         2                     959
dtype: int64

**Вывод:** чаще всего кредит не возвращают в срок те, кто не женат/не замужем, на втором месте те, кто состоит в гражданском браке, затем идут те, кто состоит в официальном браке, и те, кто в разводе. Вдовцы и вдовы - самые добросовестные заёмщики. 

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [14]:
data_income = data.pivot_table(index=['total_income_category'], values='debt', aggfunc=['sum', 'count'])
data_income['share']=data_income['sum']/data_income['count']
data_income

,sum,count,share
,debt,debt,
total_income_category,,,
A,2,25,0.080000
B,356,5041,0.070621
C,1360,16015,0.084920
D,21,350,0.060000
E,2,22,0.090909


**Вывод:** клиенты с самым высоким доходом, как ни странно, только на третьем месте по своевременному погашению кредита. Чаще всего погашают кредит вовремя клиенты с рейтингом D и B (это те, кто зарабатывает 30-50 т.р. и 200-1000 т.р. соответственно), за ними идут клиенты с рейтингом C (50-200 т.р.). Среди клиентов с низким доходом доля задолженностей выше всего (но разница с теми, кто относится к категории C, не так велика, как можно было ожидать)

#### Как разные цели кредита влияют на его возврат в срок?

In [15]:
data_purpose = data.pivot_table(index=['purpose_category'], values='debt', aggfunc=['sum', 'count'])
data_purpose['share']=data_purpose['sum']/data_purpose['count']
display(data_purpose)
data['purpose_category'].value_counts()

,sum,count,share
,debt,debt,
purpose_category,,,
операции с автомобилем,403,4306,0.093590
операции с недвижимостью,782,10811,0.072334
получение образования,370,4013,0.092200
проведение свадьбы,186,2323,0.080069


операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2323
Name: purpose_category, dtype: int64

**Вывод:** больше всего задолженностей по кредитам на операции с автомобилем и получение образования, меньше всего - по кредитам на проведение свадьбы и операции с недвижимостью. 

## Общий вывод:

Когда банк хочет оценить платёжеспособность клиента, нужно учитывать несколько параметров в совокупности, на основании каждого из них можно, например, присваивать потенциальным заёмщикам некоторое количество баллов. В целом, можно выделить следующие закономерности: 
* люди, которые состоят/ранее состояли в официальном браке, склонны более дисциплинированно платить по долгам; 
* более высокий доход не вегда имплицирует высокую вероятность выплаты кредита в срок: возможно, величина дохода, взятая сама по себе, это недостаточно информативный признак, нужно посмотреть, какую часть дохода клиент отдаёт на погашение кредитов; 
* кредиты на проведение свадьбы и получение образования чаще других не выплачиваются в срок, возможно, это связано с тем, что их берут молодые и менее ответственные заёмщики; 
* клиенты, у которых детей нет, и те, у кого их трое, чаще погашают кредит в срок (это может быть связано с тем, что за третьего ребёнка можно получить не только федеральный, но и региональный материнский капитал)